## Decision Trees

* Simple Tree like structure, model makes a decision at every node
* Useful in simple tasks
* One of the most popular algorithm
* Easy explainability, easy to show how a decision process works!

### Why decision trees are popular?

* Easy to interpret and present
* Well defined Logic, mimic human level thought
* Random Forests, Ensembles of decision trees are more powerful classifiers
* Feature values are preferred to be categorical. If the values are continuous then they are discretized prior to building the model.

### Build Decision Trees

Two common algorithms -

* CART (Classification and Regression Trees) → uses Gini Index(Classification) as metric.
* ID3 (Iterative Dichotomiser 3) → uses Entropy function and Information gain as metrics



In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('../Datasets/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
columns_to_drop=['PassengerId','Name','Ticket','Cabin','Embarked']
df=df.drop(columns_to_drop,axis=1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [5]:
le=LabelEncoder()
df['Sex']=le.fit_transform(df['Sex'])
df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500
5,0,3,1,NaN,0,0,8.4583
6,0,1,1,54.0,0,0,51.8625
7,0,3,1,2.0,3,1,21.0750
8,1,3,0,27.0,0,2,11.1333
9,1,2,0,14.0,1,0,30.0708


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [7]:
df=df.fillna(df['Age'].mean())
df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.000000,1,0,7.2500
1,1,1,0,38.000000,1,0,71.2833
2,1,3,0,26.000000,0,0,7.9250
3,1,1,0,35.000000,1,0,53.1000
4,0,3,1,35.000000,0,0,8.0500
5,0,3,1,29.699118,0,0,8.4583
6,0,1,1,54.000000,0,0,51.8625
7,0,3,1,2.000000,3,1,21.0750
8,1,3,0,27.000000,0,2,11.1333
9,1,2,0,14.000000,1,0,30.0708


In [8]:
print(df.iloc[2])

Survived     1.000
Pclass       3.000
Sex          0.000
Age         26.000
SibSp        0.000
Parch        0.000
Fare         7.925
Name: 2, dtype: float64


In [9]:
ip_col=['Pclass','Sex','Age','SibSp','Parch','Fare']
op_col=['Survived']

## Entropy 

In [10]:
def entropy(col):
    freq=np.unique(col,return_counts=True)
    N=col.shape[0]
    ent=0.0
    for i in freq[1]:
        p = i/N
        ent += -p*np.log2(p)
    return ent

## Divide Data

In [11]:
def divide_data(df,f_key,f_val):
    df_left=pd.DataFrame([],columns=df.columns)
    df_right=pd.DataFrame([],columns=df.columns)
    
    for i in range(df.shape[0]):
        val = df[f_key].iloc[i]
        
        if val < f_val:
            df_left=df_left.append(df.iloc[i])
        else:
            df_right=df_right.append(df.iloc[i])
            
    return df_left,df_right

In [12]:
a,b=divide_data(df,'Sex',df['Sex'].mean())
print(a.head())
print(b.head())

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare
1       1.0     1.0  0.0  38.0    1.0    0.0  71.2833
2       1.0     3.0  0.0  26.0    0.0    0.0   7.9250
3       1.0     1.0  0.0  35.0    1.0    0.0  53.1000
8       1.0     3.0  0.0  27.0    0.0    2.0  11.1333
9       1.0     2.0  0.0  14.0    1.0    0.0  30.0708
   Survived  Pclass  Sex        Age  SibSp  Parch     Fare
0       0.0     3.0  1.0  22.000000    1.0    0.0   7.2500
4       0.0     3.0  1.0  35.000000    0.0    0.0   8.0500
5       0.0     3.0  1.0  29.699118    0.0    0.0   8.4583
6       0.0     1.0  1.0  54.000000    0.0    0.0  51.8625
7       0.0     3.0  1.0   2.000000    3.0    1.0  21.0750


## Information Gain

In [13]:
def information_gain(df,f_key,f_val):
    left,right=divide_data(df,f_key,f_val)
    
    lp=left.shape[0]/df.shape[0]
    rp=right.shape[0]/df.shape[0]
    
    if left.shape[0]==0 or right.shape[0]==0:
        return -100000
    
    return entropy(df.Survived)-(lp*entropy(left.Survived)+rp*entropy(right.Survived))

## Overfitting

* If the tree depth is more then the learning system tightly fits the given training data and predict inaccurate outcome of untrained data.

**Methods** **to** **overcome** **overfitting** **:** 
* Post Prunning: First grow the whole tree then start removing the nodes which give poor generalization.
* Early Stopping: Stop at some early point.

## Decision Tree

In [14]:
class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.f_key=None
        self.f_val=None
        self.target=None
        self.depth=depth
        self.max_depth=max_depth
    
    def train(self,df_train):
        features=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
        info_gain=[]
        
        for i in features:
            info_gain.append(information_gain(df_train,i,df_train[i].mean()))
            
        self.f_key=features[np.argmax(info_gain)]
        self.f_val=df_train[self.f_key].mean()
                
        df_left,df_right=divide_data(df_train,self.f_key,self.f_val)
        df_left=df_left.reset_index(drop=True)
        df_right=df_right.reset_index(drop=True)
        
        if df_left.shape[0] == 0 or df_right.shape[0] == 0:
            if df_train['Survived'].mean() >= .5:
                self.target = 'Survived'
            else:
                self.target = 'Dead'
            return
        
        if self.depth >= self.max_depth:
            if df_train.Survived.mean() >= .5:
                self.target = 'Survived'
            else:
                self.target = 'Dead'
            return
        
        self.left=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(df_left)
        self.right=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(df_right)
        
        if df_train['Survived'].mean() >= .5:
            self.target = 'Survived'
        else:
            self.target = 'Dead'
        return
        
    def predict(self,df_test):
        if df_test[self.f_key]>self.f_val:
            if self.right is None:
                return self.target
            return self.right.predict(df_test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(df_test)

In [15]:
split=int(0.8*df.shape[0])
df_train=df[:split]
df_test=df[split:]
df_test=df_test.reset_index(drop=True)
print(df_train.shape,df_train.shape)
print(df_test.shape,df_test.shape)

(712, 7) (712, 7)
(179, 7) (179, 7)


In [16]:
dt=DecisionTree()
dt.train(df_train)

In [17]:
y_pred = []
for ix in range(df_test.shape[0]):
    y_pred.append(dt.predict(df_test.iloc[ix]))
y_pred = le.fit_transform(y_pred).reshape(-1,1)

In [18]:
y_actual=df_test[op_col]
y_actual=np.array(y_actual)
print("Actual  Predicted")
for i in range(10):
    print("  {}      {}".format(y_actual[i],y_pred[i])) 

Actual  Predicted
  [1]      [0]
  [0]      [0]
  [0]      [0]
  [0]      [0]
  [1]      [1]
  [1]      [1]
  [0]      [0]
  [0]      [0]
  [1]      [1]
  [0]      [0]


In [19]:
print(np.sum(y_actual == y_pred) / y_pred.shape[0])

0.8715083798882681
